In [1]:
from enum import Enum, auto
from typing import Any
import ipywidgets as widgets
from IPython.display import display
import json

from IPython.display import Javascript, display
from ipywidgets import HTML, Button, HBox, Layout, VBox, widgets


In [2]:
class DiffStatus(Enum):
    NEW = auto()
    SAME = auto()
    MODIFIED = auto()
    DELETED = auto()


background_colors = {
    DiffStatus.NEW: "#D5F1D5;",
    DiffStatus.SAME: "transparent",
    DiffStatus.MODIFIED: "#FEE9CD;",
    DiffStatus.DELETED: "#ffdddd;",
}


colors = {
    DiffStatus.NEW: "#256B24;",
    DiffStatus.SAME: "#353243",
    DiffStatus.MODIFIED: "#B8520A;",
    DiffStatus.DELETED: "#353243",
}

In [3]:
def create_diff_html(title, properties, statusses, line_length=80):
    html_str = f"<div style='font-family: monospace; width: 100%;'>{title}<br>"
    html_str += "<div style='border-left: 1px solid #B4B0BF; padding-left: 10px;'>"

    for property, status in zip(properties, statusses):
        style =f"background-color: {background_colors[status]}; color: {colors[status]}; display: block; white-space: pre-wrap; margin-bottom: 5px;"
        html_str += f"<div style='{style}'>{property}</div>"

    html_str += "</div></div>"

    return html_str

In [4]:
# TODO use ObjectDiff instead
class DiffWidget:
    def __init__(
        self,
        item_type: str,
        low_properties: list[str],
        high_properties: list[str],
        statuses: list[DiffStatus],
        is_main_widget: bool = False
    ):
        self.item_type = item_type
        self.low_properties = low_properties
        self.high_properties = high_properties
        self.statuses = statuses
        self.share_private_data=False

        self.sync: bool = False
        
        self.is_main_widget=is_main_widget
        self.widget = self._create_widget()
        
    @property
    def show_share_button(self):
        return self.item_type in ["SyftLog", "ActionObject"]
    
    @property
    def show_sync_button(self):
        return not self.is_main_widget

    @property
    def num_changes(self):
        return len([x for x in self.statuses if x != DiffStatus.SAME])

    @property
    def title(self):
        return f"{self.item_type} ({self.num_changes} changes)"

    def set_and_disable_sync(self):
        self._sync_checkbox.disabled = True
        self._sync_checkbox.value = True

    def enable_sync(self):
        self._sync_checkbox.disabled = False

    def _create_widget(self):
        html_low = create_diff_html("From", self.low_properties, self.statuses)
        html_high = create_diff_html("To", self.high_properties, self.statuses)

        diff_display_widget_old = widgets.HTML(
            value=html_low, layout=widgets.Layout(width="50%", overflow="auto")
        )
        diff_display_widget_new = widgets.HTML(
            value=html_high, layout=widgets.Layout(width="50%", overflow="auto")
        )
        content = widgets.HBox([diff_display_widget_old, diff_display_widget_new])
        
        checkboxes = []

        
        if self.show_sync_button:
            self._sync_checkbox = widgets.Checkbox(
                value=self.sync,
                description="Sync",
            )
            self._sync_checkbox.observe(self._on_sync_change, 'value')
            checkboxes.append(self._sync_checkbox)

        
        if self.show_share_button:
            self._share_private_checkbox = widgets.Checkbox(
                value=self.share_private_data,
                description="Share private data",
            )
            self._share_private_checkbox.observe(self._on_share_private_data_change, 'value')
            checkboxes = [self._share_private_checkbox] + checkboxes
            
        checkboxes_widget = widgets.HBox(checkboxes)
        kwargs = {}
        if self.is_main_widget:
            kwargs["layout"] = Layout(border='#353243 solid 0.5px', padding="16px")
            
        widget = widgets.VBox([checkboxes_widget, content], **kwargs)
        return widget

    def _on_sync_change(self, change):
        self.sync = change['new']

    def _on_share_private_data_change(self, change):
        self.share_private_data = change['new']

In [5]:
fn_str = """@syft.function
def compute_mean(data: np.ndarray):
    print("computing mean...")
    return data.mean()
"""

In [6]:
code_diff = DiffWidget(
    item_type="UserCode",
    low_properties=[
        "status: pending",
        "approved_by: None",
        "date_created: 2024-04-27",
        "code: None",
    ],
    high_properties=[
        "status: approved",
        "approved_by: admin",
        "date_created: 2024-04-27",
        f"code: {fn_str}",
    ],
    statuses=[
        DiffStatus.MODIFIED,
        DiffStatus.MODIFIED,
        DiffStatus.SAME,
        DiffStatus.NEW,
    ],
    is_main_widget=True
)


In [7]:
request_diff = DiffWidget(
    item_type="Request",
    low_properties=[
        "status: pending",
        "approved_by: None",
        "date_created: 2024-04-27",
    ],
    high_properties=[
        "status: approved",
        "approved_by: admin",
        "date_created: 2024-04-27",
    ],
    statuses=[
        DiffStatus.MODIFIED,
        DiffStatus.MODIFIED,
        DiffStatus.SAME,
    ],
)


In [8]:
main_widget = code_diff
batch_widgets = [request_diff]
display(code_diff.widget)

In [9]:
code_diff.widget

In [10]:
print(code_diff.share_private_data, code_diff.sync)

False False


In [11]:
request_diff.show_sync_button

True

In [12]:
request_diff.set_and_disable_sync()
print(request_diff.share_private_data, request_diff.sync)

False True


In [13]:
request_diff.enable_sync()

In [14]:


def copy_text_button(text: str) -> widgets.Widget:
    button = widgets.Button(
        icon="clone",
        # style={'button_color': 'transparent'},
        layout=widgets.Layout(width="25px", height="25px", margin="0", padding="0"),
    )
    output = widgets.Output(layout=widgets.Layout(display="none"))
    copy_js = Javascript(f"navigator.clipboard.writeText({json.dumps(text)})")

    def on_click(_: widgets.Button) -> None:
        output.clear_output()
        with output:
            display(copy_js)

    button.on_click(on_click)

    return widgets.Box(
        (button, output), layout=widgets.Layout(display="flex", align_items="center")
    )

def create_item_type_box(item_type):
    # TODO different bg for different types (levels?)
    style = (
        "background-color: #C2DEF0; "
        "border-radius: 4px; "
        "padding: 4px 6px; "
        "color: #373B7B;"
    )
    return HTML(
        value=f"<span style='{style}'>{item_type}</span>",
        layout=Layout(margin="0 5px 0 0")
    )

def create_name_id_label(item_name, item_id):
    item_id_short = item_id[:4] + '...' if len(item_id) > 4 else item_id
    return HTML(
        value=(
            f"<span style='margin-left: 5px; font-weight: bold; color: #373B7B;'>{item_name}</span> "
            f"<span style='margin-left: 5px; color: #B4B0BF;'>#{item_id_short}</span>"
        )
    )

class HeaderWidget:
    def __init__(
        self,
        item_type: str,
        item_name: str,
        item_id: str,
        num_diffs: int,
        source_side: str,
        target_side: str,
    ):
        self.item_type = item_type
        self.item_name = item_name
        self.item_id = item_id
        self.num_diffs = num_diffs
        self.source_side = source_side
        self.target_side = target_side
        self.widget = self.create_widget()

    def create_widget(self):
        type_box = create_item_type_box(self.item_type)
        name_id_label = create_name_id_label(self.item_name, self.item_id)
        copy_button = copy_text_button(self.item_id)

        first_line = HTML(value="<span style='color: #B4B0BF;'>Syncing changes on</span>")
        second_line = HBox(
            [type_box, name_id_label, copy_button], layout=Layout(align_items="center")
        )
        third_line = HTML(
            value=f"<span style='color: #5E5A72;'>This would sync <span style='color: #B8520A'>{self.num_diffs} changes </span> from <i>{self.source_side} Node</i> to <i>{self.target_side} Node</i></span>"
        )
        fourth_line = HTML(
            value=f"<div style='height: 16px;'></div>"
        )
        header = VBox([first_line, second_line, fourth_line])
        return header


header_widget = HeaderWidget(
    item_type="Job",
    item_name="main_1",
    item_id="12345678",
    num_diffs=22,
    source_side="Low",
    target_side="High",
)

header_widget.widget

In [15]:

separator = widgets.HTML(
    value='<div style="text-align: center; margin: 10px 0; border: 1px dashed #B4B0BF;"></div>',
    layout=Layout(width="100%"),
)

separator

HTML(value='<div style="text-align: center; margin: 10px 0; border: 1px dashed #B4B0BF;"></div>', layout=Layou…

In [16]:
header_widget = HeaderWidget(
    item_type="Code",
    item_name="compute_mean",
    item_id="12345678",
    num_diffs=2,
    source_side="Low",
    target_side="High",
)

main_accordion = widgets.Accordion(
    children=[d.widget for d in batch_widgets], titles=[d.title for d in batch_widgets]
)
dependencies_accordion = widgets.Accordion(
    children=[d.widget for d in batch_widgets], titles=[d.title for d in batch_widgets]
)
full_widget = widgets.VBox([header_widget.widget,
                            main_widget.widget,
                            main_accordion,
                            separator,
                            dependencies_accordion])
# full_widget = widgets.VBox([header_widget.widget, main_accordion, separator, dependencies_accordion])
display(full_widget)

# TODO

- integrate with ObjectDiff / ObjectDiffBatch
- set diff_widget.set_and_disable_sync based on dependencies
- expand button for long properties (figma log)
- checkboxes in accordion header
- styling
- 